# DIY Intent based networking

The objective of this module is to show you a few things:

1. How to create your own "complex" tasks by leveraging existing tasks
2. How complex workflows become simple thanks to the power of python

To do that, we are going to create a task that synchronizes users; it will add missing users, modify the ones that need to be modified and remove old users. This task will be platform agnostic and will support even platforms without hierarchical config.

As usual, let's initializate the nornir object:

In [1]:
from nornir import InitNornir
nr = InitNornir(
    inventory={
        "options": {
            "host_file": "5_manage_users/inventory/hosts.yaml",
            "group_file": "5_manage_users/inventory/groups.yaml",
            "defaults_file": "5_manage_users/inventory/defaults.yaml",
        }
    }
)

We are going to store the users in a yaml file different than the inventory:

In [2]:
!cat 5_manage_users/data/users.yaml

---
joe:
jane:
admin:


Now we are going to create a template per platform that takes:
    1. `remove_users` - users we want to remove
    2. `desired_users` - users we want in the system

In [3]:
!cat 5_manage_users/templates/eos/users.j2

{% for user in remove_users %}
no username {{ user }}
{% endfor %}

{% for user in desired_users %}
username {{ user }} privilege 15 role network-admin secret sha512 $6$KxgYSk1jaMjZxxBs$1Qmn8UKx7rj3tREiXrdbhXKCC2Mw0w7LwysWCY/xjAX3QDOe0whVdSbn6O50pCsAhNAqB3pk9kY0Nw2OB/Uhz1
{% endfor %}


In [4]:
!cat 5_manage_users/templates/junos/users.j2

{% for user in remove_users %}
delete system login user {{ user }}
{% endfor %}

{% for user in desired_users %}
set system login user {{ user }} uid 2000
set system login user {{ user }} class super-user
{% endfor %}



First, we create a function that does what we need:

In [5]:
from nornir.plugins.tasks.networking import napalm_configure, napalm_get
from nornir.plugins.tasks.text import template_file

def manage_users(task, desired_users):
    # get users from device
    state_users = task.run(task=napalm_get,
                           getters=["users"],
                           severity_level=logging.DEBUG)

    # let's verify if the users we got are in desired_users
    # if they are not we have to remove them
    users_to_remove = []
    for user in state_users.result["users"]:
        if user not in desired_users:
            users_to_remove.append(user)
    
    # we render the template for the platform passing desired_users and users_to_remove
    users_config = task.run(task=template_file,
                            path=f"5_manage_users/templates/{task.host.platform}",
                            template="users.j2",
                            desired_users=desired_users,
                            remove_users=users_to_remove,
                            severity_level=logging.DEBUG)
    
    # we load the resulting configuration into the device
    task.run(task=napalm_configure,
             configuration=users_config.result)


In [6]:
import logging

import ruamel.yaml

from nornir.plugins.functions.text import print_result
    
# we load from a yaml file the users we want
yaml = ruamel.yaml.YAML()
with open("5_manage_users/data/users.yaml", "r") as f:
    desired_users = yaml.load(f.read())

spines = nr.filter(role="spine")

# we call manage_users passing the users we loaded from the yaml file
r = spines.run(task=manage_users,
               desired_users=desired_users)

In [7]:
print_result(r)

manage_users********************************************************************
* spine00.bma ** changed : True ************************************************
vvvv manage_users ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
---- napalm_configure ** changed : True ---------------------------------------- INFO
@@ -14,8 +14,9 @@
 !
 aaa root secret sha512 $6$XoN/NKq7rqSxD9ov$xeiYtli3w.fEiNCsYQ4aJkWWZCor6tNVOwaTwB0UinZTTtV0qBk0VvNBlR4Kcsctbf6atsrKt3iTLc9wzOqoT1
 !
-username bob privilege 15 role network-admin secret sha512 $6$6ToHoIeHQFIQeBTL$2iEVU6ReAVTNaoaxdSawgsa2e/dOxclhMb2D7X7vKYpNKmsXRyEs6YhTxij24i10d.IozA5cKVok5zm2lmV0v/
+username joe privilege 15 role network-admin secret sha512 $6$KxgYSk1jaMjZxxBs$1Qmn8UKx7rj3tREiXrdbhXKCC2Mw0w7LwysWCY/xjAX3QDOe0whVdSbn6O50pCsAhNAqB3pk9kY0Nw2OB/Uhz1
+username jane privilege 15 role network-admin secret sha512 $6$KxgYSk1jaMjZxxBs$1Qmn8UKx7rj3tREiXrdbhXKCC2Mw0w7LwysWCY/xjAX3QDOe0whVdSbn6O50pCsAhNAqB3pk9kY0Nw2OB/Uhz1
 !
 inter